In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

def plot_data(config):
    df = pd.read_csv(config['csv_file'])
    df["csvTime"] = pd.to_datetime(df["csvTime"])

    if config.get('start_time'):
        df = df[df["csvTime"] >= pd.to_datetime(config['start_time'])]
    if config.get('end_time'):
        df = df[df["csvTime"] <= pd.to_datetime(config['end_time'])]

    if 'data_filters' in config:
        for filter_condition in config['data_filters']:
            df = df.query(filter_condition)

    _, ax = plt.subplots(figsize=(14, 7))

    for series in config['series']:
        ax.plot(df["csvTime"], df[series['column']], label=series['label'], linestyle=series['linestyle'], color=series['color'])

    for _, row in df.iterrows():
        time_str = row['csvTime'].strftime('%H:%M:%S')
        
        for scatter in config.get('scatter_conditions', []):
            if scatter['condition'](row):
                ax.scatter(row['csvTime'], row[scatter['column']], color=scatter['color'], s=scatter['size'], edgecolors="black", zorder=3)
                ax.annotate(f"{row[scatter['label']]}\n({time_str})",
                            (row['csvTime'], row[scatter['column']]),
                            textcoords="offset points", xytext=scatter['xytext'],
                            ha='center', fontsize=scatter['fontsize'], color=scatter['color'],
                            fontweight="bold", bbox=dict(facecolor='white', alpha=0.7))

    ax.xaxis.set_major_formatter(mdates.DateFormatter(config['x_label_format']))
    ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=5))
    plt.xticks(rotation=45)

    ax.set_xlabel(config['x_label'], fontsize=12, fontweight="bold")
    ax.set_ylabel(config['y_label'], fontsize=12, fontweight="bold")
    ax.set_title(config['title'], fontsize=14, fontweight="bold")
    ax.legend(fontsize=12)
    plt.grid(True, linestyle="--", alpha=0.6)

    plt.show()



In [ ]:
# 'start_time': "2024-08-27 7:50:00",
# 'end_time': "2024-08-27 9:00:00",
config_ajia = {
    'csv_file': "../data/Ajia_plc_1.csv",
    'start_time': "2024-05-25 7:50:00",
    'end_time': "2024-05-25 9:00:00",
    'x_label_format': '%H:%M:%S',
    'x_label': "时间",
    'y_label': "电流",
    'title': "A架数据分析",
    'series': [
        {'column': 'Ajia-3_v', 'label': "Ajia-3_v (1启动柜电流)", 'linestyle': "-", 'color': sns.color_palette("husl", 2)[0]},
        {'column': 'Ajia-5_v', 'label': "Ajia-5_v (2启动柜电流)", 'linestyle': "-", 'color': sns.color_palette("husl", 2)[1]}
    ],
    'scatter_conditions': [
        {'condition': lambda row: row['status'] != 'False', 'column': 'Ajia-3_v', 'color': 'red', 'size': 50, 'xytext': (0, 10), 'fontsize': 10, 'label': "status"},
        {'condition': lambda row: row['check_current_presence'] != 'False', 'column': 'Ajia-5_v', 'color': 'blue', 'size': 50, 'xytext': (0, -20), 'fontsize': 10, 'label': "check_current_presence"}
    ]
}

plot_data(config_ajia)


In [ ]:
config_ajia_angle = {
    'csv_file': "../data/Ajia_plc_1.csv",
    'start_time': "2024-08-23 6:00:00",
    'end_time': "2024-08-23 11:30:00",
    'x_label_format': '%H:%M:%S',
    'x_label': "时间",
    'y_label': "角度 (°)",
    'title': "A架角度分析",
    'series': [
        {'column': 'Ajia-0_v', 'label': "Ajia-0_v (A架右舷角度)", 'linestyle': "-", 'color': sns.color_palette("husl", 2)[0]},
        {'column': 'Ajia-1_v', 'label': "Ajia-1_v (A架左舷角度)", 'linestyle': "-", 'color': sns.color_palette("husl", 2)[1]},
        {'column': 'angle_range', 'label': "Angle Range", 'linestyle': "--", 'color': "green"}
    ],
    'scatter_conditions': [
        {'condition': lambda row: row['status'] != 'False', 'column': 'Ajia-0_v', 'color': 'red', 'size': 100, 'xytext': (0, 10), 'fontsize': 12, 'label': "status"},
    ]
}

plot_data(config_ajia_angle)

In [ ]:
config_dp = {
    'csv_file': "../data/Port3_ksbg_9.csv",
    'start_time': "2024-08-23 8:00:00",
    'end_time': "2024-08-23 11:00:00",
    'x_label_format': '%H:%M:%S',
    'x_label': "时间",
    'y_label': "功率 (kW)",
    'title': "DP 系统数据分析",
    'data_filters': [
        'P3_33 != -1.0', 'P3_18 != -1.0'
    ],
    'series': [
        {'column': 'P3_33', 'label': "P3_33 (艏推启动功率允许)", 'linestyle': "-", 'color': sns.color_palette("husl", 2)[0]},
        {'column': 'P3_18', 'label': "P3_18 (艏推功率反馈)", 'linestyle': "-", 'color': sns.color_palette("husl", 2)[1]}
    ],
    'scatter_conditions': [
        {'condition': lambda row: row['status'] != 'False', 'column': 'P3_33', 'color': 'green', 'size': 100, 'xytext': (0, 10), 'fontsize': 12, 'label': "status"},
    ]
}

plot_data(config_dp)

In [ ]:
config_crane = {
    'csv_file': "../data/device_13_11_meter_1311.csv",
    'start_time': "2024-08-23 18:00:00",
    'end_time': "2024-08-23 20:00:00",
    'x_label_format': '%H:%M:%S',
    'x_label': "时间",
    'y_label': "有功功率 (kW)",
    'title': "折臂吊车数据分析",
    'series': [
        {'column': '13-11-6_v', 'label': "13-11-6_v (折臂吊车液压-Pt有功功率)", 'linestyle': "-", 'color': sns.color_palette("husl", 2)[0]},
    ],
    'scatter_conditions': [
        {'condition': lambda row: row['status'] != 'False', 'column': '13-11-6_v', 'color': 'green', 'size': 100, 'xytext': (0, 10), 'fontsize': 12, 'label': "status"},
    ]
}

plot_data(config_crane)